<font size="6"><center>**Walk-through**</font>

This notebooks attemps to guide newcomer quants in their first steps to become rich

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run ../nb_config.py

running notebook configuration


In [3]:
import numpy as np
import pandas as pd
import os

# Parameters

In [4]:
start_dt = pd.Timestamp('2015-01-01')
end_dt = pd.Timestamp('2017-12-31')
split_dt = pd.Timestamp('2017-07-31')
tau = 5

In [5]:
ticker = 'AMZN'

# Load Data

`folder structure:`  
|-notebooks/  
|-data/  
|--raw/  
|---wiki_prices.csv

In [6]:
wiki_prices_path = os.path.join("..", "data", "raw", "wiki_prices.csv")  #
wiki_prices = pd.read_csv(wiki_prices_path, parse_dates=['date'], index_col=['ticker', 'date'])

In [7]:
pd.concat([wiki_prices.head(2), wiki_prices.tail(2)], axis=0)

open    high     low   close          volume  \
ticker date                                                         
A      1999-11-18 45.5000 50.0000 40.0000 44.0000 44,739,900.0000   
       1999-11-19 42.9400 43.0000 39.8100 40.3800 10,897,100.0000   
ZUMZ   2018-03-26 23.7500 24.8000 23.7000 24.6500    375,320.0000   
       2018-03-27 24.6500 24.6500 23.3500 23.6000    403,884.0000   

                   ex-dividend  split_ratio  adj_open  adj_high  adj_low  \
ticker date                                                                
A      1999-11-18       0.0000       1.0000   31.0420   34.1120  27.2896   
       1999-11-19       0.0000       1.0000   29.2954   29.3363  27.1600   
ZUMZ   2018-03-26       0.0000       1.0000   23.7500   24.8000  23.7000   
       2018-03-27       0.0000       1.0000   24.6500   24.6500  23.3500   

                   adj_close      adj_volume  
ticker date                                   
A      1999-11-18    30.0186 44,739,900.0000  
       1999-11-19    27.5489 10,897,100.0000  
ZUMZ   2018-03-26    24.6500    375,320.0000  
       2018-03-27    23.6000    403,884.0000

In [8]:
wiki_prices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZUMZ', Timestamp('2018-03-27 00:00:00'))
Columns: 12 entries, open to adj_volume
dtypes: float64(12)
memory usage: 1.4+ GB


In [9]:
wiki_meta_path = os.path.join("..", "data", "raw", "us_equities_meta_data.csv")  #
wiki_prices_meta = pd.read_csv(wiki_meta_path, index_col='ticker')

wiki_prices_meta.head()

,name,lastsale,marketcap,ipoyear,sector,industry
ticker,,,,,,
PIH,"1347 Property Insurance Holdings, Inc.",7.2001,"43,090,000.0000","2,014.0000",Finance,Property-Casualty Insurers
PIHPP,"1347 Property Insurance Holdings, Inc.",25.6200,NaN,NaN,Finance,Property-Casualty Insurers
TURN,180 Degree Capital Corp.,2.2600,"70,330,000.0000",NaN,Finance,Finance/Investors Services
FLWS,"1-800 FLOWERS.COM, Inc.",12.9000,"833,390,000.0000","1,999.0000",Consumer Services,Other Specialty Stores
FCCY,1st Constitution Bancorp (NJ),21.3000,"178,140,000.0000",NaN,Finance,Savings Institutions


# Data Preparation

In [10]:
# use a rule to filter tickers of interest
wiki_smpl = (wiki_prices
             .reset_index()
             .merge(wiki_prices_meta['sector'], on='ticker', how='left')
             .set_index(['ticker', 'date'])
            )
wiki_smpl['sector'] = wiki_smpl['sector'].fillna('NA')

## Targets

In [11]:
def log_returns(prices: pd.Series, tau: int) -> pd.Series:
    """
    Compute log returns on time series data
    log(a/b) = log(a) - log(b)
    @param prices: time series data
    @param tau: differencing period
    @return: returns
    """
    return np.log(prices).diff(tau)

In [12]:
log_rets = (wiki_smpl
          .groupby('ticker')['adj_close']
          .transform(lambda grp: log_returns(grp, tau=-tau))
          )

In [13]:
targets = -1*log_rets
targets.name = 'target'

## Features

Features are also called **Alpha factors** (See S2 slides) and are computed on past data
Some features are based on returns (like targets, but are backwards returns), other on prices and other features on alternative data (sentiment analysis in financial analysis reports,)

In [14]:
feat_1yr_mom = (wiki_smpl
            .groupby('ticker')['adj_close']
            .pct_change(252)
               )
feat_1yr_mom.name = 'x_mon_1yr'

In [15]:
feat_5d_mom = (wiki_smpl
            .groupby('ticker')['adj_close']
            .apply(lambda grp: log_returns(grp, tau=5))
              )
feat_5d_mom.name = 'x_mon_5d'

In [16]:
feat_close_vs_open = wiki_smpl['adj_close']-wiki_smpl['adj_open']
feat_close_vs_open.name = 'x_close_vs_open'

## Join and Filter

In [17]:
features_and_targets = pd.concat([targets, wiki_smpl['sector'], feat_1yr_mom, feat_5d_mom, feat_close_vs_open], axis=1)

dates = features_and_targets.index.get_level_values('date')
mask_dates = (start_dt <=dates) & (dates <= end_dt)

features_and_targets = (
    features_and_targets
    .loc[mask_dates
         & (features_and_targets['sector'] != 'NA')
    ]
)
# TODO: filtering the entire population is needed in order to:
# get a subset of time
# a sample of stocks that are tradable, many options like highly traded stocks or stocks in SP500 list

In [18]:
features_and_targets.head(2)

target         sector  x_mon_1yr  x_mon_5d  x_close_vs_open
ticker date                                                                   
A      2015-01-02  0.0007  Capital Goods     0.0079   -0.0115          -0.6028
       2015-01-05  0.0078  Capital Goods    -0.0234   -0.0365          -0.5056

# Feature Enginering

In [19]:
features_and_targets.corr('spearman')

,target,x_mon_1yr,x_mon_5d,x_close_vs_open
target,1.0000,0.0009,-0.0231,-0.0135
x_mon_1yr,0.0009,1.0000,0.1242,0.0457
x_mon_5d,-0.0231,0.1242,1.0000,0.3232
x_close_vs_open,-0.0135,0.0457,0.3232,1.0000


In [20]:
features_and_targets['x_mon_1yr_dm'] = (
    features_and_targets
    .groupby(['sector', 'date'])['x_mon_1yr']
    .transform(lambda grp: grp-grp.mean())
) 


In [21]:
features_and_targets[['x_mon_1yr_dm', 'target']].corr('spearman')

,x_mon_1yr_dm,target
x_mon_1yr_dm,1.0000,0.0138
target,0.0138,1.0000


In [22]:
# you can use other metrics like accuracy,
# or translate it to weights and then compute a pnl

pd.crosstab(
    index=features_and_targets['x_mon_1yr_dm']>=0,
    columns=features_and_targets['target']>=0,
    normalize=True
)

target,False,True
x_mon_1yr_dm,,
False,0.2513,0.2756
True,0.2182,0.2549


In [23]:
# * is eval metric consistent over time?
# * is eval metric consistent over tickers
# Preliminary stage of feature building and refinement can be done in train
# if there is no future data involved on calculations/selections
# final feature selection and feature engineering techniques,
# should be done only on train.
# In this case, sector neautralization (demeaning) can be considered part of the basic feature computation,
# comparion is only shown for demostrative purposes